<a href="https://colab.research.google.com/github/etmcrae/DOGE-Clone/blob/main/DOGE_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


```
** Department of Government Efficiency (DOGE) API ** **bold text**
```



**Contains Savings & Payments**

Savings include: grants, contracts, leases

Payments include: payments, statistics

Source is https://doge.gov/savings

**From https://github.com/m-nolan/doge-scrape/blob/main/doge-scrape.py**

might need to update the USAspending data from https://api.usaspending.gov/docs/endpoints

In [1]:
import os
from datetime import datetime
from time import sleep

import numpy as np
import pandas as pd
import requests as req
!pip install validators
!pip install ratelimit # Install the ratelimit package
from bs4 import BeautifulSoup
from ratelimit import limits, sleep_and_retry
!pip install selenium
from selenium.webdriver import Firefox
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from tqdm import tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ratelimit: filename=ratelimit-2.2.1-py3-none-any.whl size=5893 sha256=a01c907d675dbbb8656d50e7c4977ec29373c0bd0ca7943cc42256f354e8b167
  Stored in directory: /root/.cache/pip/wheels/ee/d5/e5/8fbffe089140fb498987b7709becf861086daace105d243475
Successfully built ratelimit
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.14.0
    Uninstalling typing_extensions-4.14.0:
      Successfully uninstalled typing_extensions-4.14.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following de

In [2]:
!wget https://files.usaspending.gov/award_data_archive/FY2025_All_Contracts_Full_20250506.zip
!unzip FY2025_All_Contracts_Full_20250506.zip

--2025-06-06 19:24:50--  https://files.usaspending.gov/award_data_archive/FY2025_All_Contracts_Full_20250506.zip
Resolving files.usaspending.gov (files.usaspending.gov)... 166.123.8.118, 2610:108:4100:100c::9:168
Connecting to files.usaspending.gov (files.usaspending.gov)|166.123.8.118|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 739753150 (705M) [application/zip]
Saving to: ‘FY2025_All_Contracts_Full_20250506.zip’

FY2025_All_Contract 100%[===================>] 705.48M  48.6MB/s    in 18s     

2025-06-06 19:25:09 (39.1 MB/s) - ‘FY2025_All_Contracts_Full_20250506.zip’ saved [739753150/739753150]

Archive:  FY2025_All_Contracts_Full_20250506.zip
  inflating: FY2025_All_Contracts_Full_20250508_1.csv  
  inflating: FY2025_All_Contracts_Full_20250508_2.csv  
  inflating: FY2025_All_Contracts_Full_20250508_3.csv  


In [3]:
!ls -la


total 5916284
drwxr-xr-x 1 root root       4096 Jun  6 19:25 .
drwxr-xr-x 1 root root       4096 Jun  6 19:23 ..
drwxr-xr-x 4 root root       4096 Jun  5 13:38 .config
-rw-r--r-- 1 root root  739753150 May 13 19:08 FY2025_All_Contracts_Full_20250506.zip
-rw-r--r-- 1 root root 2179860288 May  9 01:52 FY2025_All_Contracts_Full_20250508_1.csv
-rw-r--r-- 1 root root 2099189096 May  9 01:53 FY2025_All_Contracts_Full_20250508_2.csv
-rw-r--r-- 1 root root 1039433226 May  9 01:54 FY2025_All_Contracts_Full_20250508_3.csv
drwxr-xr-x 1 root root       4096 Jun  5 13:38 sample_data


In [ ]:
!unzip -l ../FY2025_All_Contracts_Full_20250506.zip
!unzip ../FY2025_All_Contracts_Full_20250506.zip
!ls -la
df1 = pd.read_csv('./FY2025_All_Contracts_Full_20250508_1.csv')
df2 = pd.read_csv('./FY2025_All_Contracts_Full_20250508_2.csv')
df3 = pd.read_csv('./FY2025_All_Contracts_Full_20250508_3.csv')


full_contracts = pd.concat([df1, df2, df3], ignore_index=True)

# Optionally, remove duplicate rows if they exist across files
# full_contracts.drop_duplicates(inplace=True)

print(full_contracts.head())
full_contracts.info()

unzip:  cannot find or open ../FY2025_All_Contracts_Full_20250506.zip, ../FY2025_All_Contracts_Full_20250506.zip.zip or ../FY2025_All_Contracts_Full_20250506.zip.ZIP.
unzip:  cannot find or open ../FY2025_All_Contracts_Full_20250506.zip, ../FY2025_All_Contracts_Full_20250506.zip.zip or ../FY2025_All_Contracts_Full_20250506.zip.ZIP.
total 5916284
drwxr-xr-x 1 root root       4096 Jun  6 19:25 .
drwxr-xr-x 1 root root       4096 Jun  6 19:23 ..
drwxr-xr-x 4 root root       4096 Jun  5 13:38 .config
-rw-r--r-- 1 root root  739753150 May 13 19:08 FY2025_All_Contracts_Full_20250506.zip
-rw-r--r-- 1 root root 2179860288 May  9 01:52 FY2025_All_Contracts_Full_20250508_1.csv
-rw-r--r-- 1 root root 2099189096 May  9 01:53 FY2025_All_Contracts_Full_20250508_2.csv
-rw-r--r-- 1 root root 1039433226 May  9 01:54 FY2025_All_Contracts_Full_20250508_3.csv
drwxr-xr-x 1 root root       4096 Jun  5 13:38 sample_data


<ipython-input-7-5cbc6ca64e50>:4: DtypeWarning: Columns (5,47,68,71,72,81,107,108,115,116,117,118,135,136,164,165,171,172,179,180) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('./FY2025_All_Contracts_Full_20250508_1.csv')


**Savings**

In [ ]:
# grants

In [ ]:
#contracts

In [ ]:
#leases

**Payments**

In [ ]:
#payments

In [ ]:
#statistics

GITHUB

In [ ]:
# prompt: import data from url and view as data frame

import pandas as pd

# Define the URL of the CSV file
url = 'https://github.com/etmcrae/DOGE-Clone/blob/main/data/doge-contract.csv'

# Read the CSV data from the URL into a pandas DataFrame
df = pd.read_csv(url)

# Display the first few rows of the DataFrame
print(df.head())

ParserError: Error tokenizing data. C error: Expected 1 fields in line 42, saw 61


In [ ]:
import requests

url = 'https://github.com/etmcrae/DOGE-Clone/blob/main/data/doge-contract.csv'
url.split('/')[-1]

print(content)









<!DOCTYPE html>
<html
  lang="en"
  
  data-color-mode="auto" data-light-theme="light" data-dark-theme="dark"
  data-a11y-animated-images="system" data-a11y-link-underlines="true"
  
  >



  <head>
    <meta charset="utf-8">
  <link rel="dns-prefetch" href="https://github.githubassets.com">
  <link rel="dns-prefetch" href="https://avatars.githubusercontent.com">
  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">
  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">
  <link rel="preconnect" href="https://github.githubassets.com" crossorigin>
  <link rel="preconnect" href="https://avatars.githubusercontent.com">

  


  <link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/light-74231a1f3bbb.css" /><link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/light_high_contrast-83beb16e0ecf.css" /><link crossorigin="anonymous" media="all" rel="